In [182]:
import asyncio
import csv
import json
import logging
from collections import defaultdict
from datetime import datetime
import websockets
from binance.client import Client
from binance.enums import *
from binance.enums import TIME_IN_FORCE_GTC, TIME_IN_FORCE_IOC, TIME_IN_FORCE_FOK
from binance.exceptions import BinanceAPIException

import time

import logging
from urllib.parse import urlencode
import hmac
import hashlib
import requests

import pandas as pd

from binance.um_futures import UMFutures
import logging
import math
from datetime import datetime, timezone
import plotly.graph_objects as go

In [183]:
import ccxt

### Spot Keys

In [184]:
api_key_spot = 'uo2zeqvr50AnBX3tyQL97Y5ZrDoBPLsqyRlydPfE3J5kDIzYr4nd9twvezzjsmMW'

api_secret_spot = 'l1J4chf5drzeYjvW1tbKg0AiYyQ6qXLsdfGFTpLc79kVtMlZLHVvW595PxCXe2Fj'


In [185]:
client = Client(api_key_spot, api_secret_spot, testnet= True)

In [186]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1751025138446,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '342396.45862374', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'l

### Futures

In [187]:
# api_key = '13c9017fce28a8d9fd51a8b85f52af046cd80a674e69c8e291dfb4565cf2bbd1'
# api_secret = '77a80276c0f8f1eeff57490121bf4cfeb5dcc97650c1f1ed031fdbe467a81195'

In [188]:
api_key = '7c16d1473c07bd83d41d445ce2fddb9ad9ef15cd2ab6c7253c22c5a46d76a180'
api_secret = '7a35c29fe03aa2e6692a7ecb9672d1dc8661f0778c1b707187d990be1a26f242'

In [189]:
exchange = ccxt.binance({'apiKey': api_key,
                        'secret': api_secret,
                        'options':{'defaultType':'future'}}
                        )

In [190]:
exchange.set_sandbox_mode(True)

In [191]:
exchange.fetch_balance()

{'info': {'totalInitialMargin': '52.66012550',
  'totalMaintMargin': '6.31921506',
  'totalWalletBalance': '14999.55947387',
  'totalUnrealizedProfit': '48.11280386',
  'totalMarginBalance': '15047.67227773',
  'totalPositionInitialMargin': '52.66012550',
  'totalOpenOrderInitialMargin': '0.00000000',
  'totalCrossWalletBalance': '14999.55947387',
  'totalCrossUnPnl': '48.11280386',
  'availableBalance': '14992.02970223',
  'maxWithdrawAmount': '14992.02970223',
  'assets': [{'asset': 'FDUSD',
    'walletBalance': '0.00000000',
    'unrealizedProfit': '0.00000000',
    'marginBalance': '0.00000000',
    'maintMargin': '0.00000000',
    'initialMargin': '0.00000000',
    'positionInitialMargin': '0.00000000',
    'openOrderInitialMargin': '0.00000000',
    'crossWalletBalance': '0.00000000',
    'crossUnPnl': '0.00000000',
    'availableBalance': '0.00000000',
    'maxWithdrawAmount': '0.00000000',
    'updateTime': '0'},
   {'asset': 'BFUSD',
    'walletBalance': '0.00000000',
    'unr

In [192]:
# api_key = '7fb4c6b8c89aa920a5aa83ca4e1b05a0be2a2fad5c6bb6cad41ada4e14b3116f'
# api_secret = '28705594b73c8262168c99c327a03b3016f1e2977a1bef577606501e74d285fa'

client = Client(api_key, api_secret)
client.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'

BASE_URL = 'https://testnet.binancefuture.com'

### Current Spot and Futures margin

In [193]:
def create_signature(query_string, secret):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def get_futures_available_balance(asset='USDT'):
    endpoint = '/fapi/v2/balance'  # returns list of assets with balances
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp}'
    signature = create_signature(query_string, api_secret)
    headers = {'X-MBX-APIKEY': api_key}
    url = f'{BASE_URL}{endpoint}?{query_string}&signature={signature}'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        balances = response.json()
        for bal in balances:
            if bal['asset'] == asset:
                print(f"Available {asset} balance (margin): {bal['availableBalance']}")
                return float(bal['availableBalance'])
        print(f"{asset} balance not found.")
    else:
        print(f"Error fetching futures balance: {response.status_code}, {response.text}")

if __name__ == '__main__':
    futures_balance = get_futures_available_balance()

Available USDT balance (margin): 14992.02970223


In [194]:
SPOT_BASE_URL = 'https://testnet.binance.vision'

def get_spot_balance(asset='USDT'):
    endpoint = '/api/v3/account'
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp}'
    signature = create_signature(query_string, api_secret_spot)
    headers = {'X-MBX-APIKEY': api_key_spot}
    url = f'{SPOT_BASE_URL}{endpoint}?{query_string}&signature={signature}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        for bal in data['balances']:
            if bal['asset'] == asset:
                print(f"Spot {asset} free balance: {bal['free']}")
                return float(bal['free'])
        print(f"{asset} not found in spot balances.")
    else:
        print(f"Error fetching spot balance: {response.status_code}, {response.text}")

if __name__ == '__main__':
    spot_balance = get_spot_balance()

Spot USDT free balance: 342396.45862374


In [195]:
INITIAL_CAPITAL = min(futures_balance, spot_balance)
INITIAL_CAPITAL

14992.02970223

### Data Handling

In [196]:
# 
logging.basicConfig(
    format="%(asctime)s [%(levelname)-5s] %(message)s",
    level=logging.INFO,
)

SYMBOLS_CSV = "binancespot.csv"
# UPDATE_INTERVAL = 5      # Seconds between display updates
# RECONNECT_DELAY = 5      # Seconds between socket reconnects

# ────────────────────────── DATA INIT ──────────────────────────────
with open(SYMBOLS_CSV) as f:
    reader = csv.DictReader(f)
    symbols = [row["spot_symbol"].lower() for row in reader]

def create_combined_url(base_url: str, streams: list) -> str:
    return f"{base_url}/stream?streams={'/'.join(streams)}"

stream_config = {
    "spot":      (symbols, "wss://stream.binance.com:9443", "@bookTicker"),
    "perp_book": (symbols, "wss://fstream.binance.com",      "@bookTicker"),
    "perp_mark": (symbols, "wss://fstream.binance.com",      "@markPrice"),
}

stream_urls = {
    k: create_combined_url(base, [f"{s}{suffix}" for s in syms])
    for k, (syms, base, suffix) in stream_config.items()
}

market_data = defaultdict(lambda: {
    "spot_bid": None, "spot_ask": None,
    "perp_bid": None, "perp_ask": None,
    "funding":  None, "mark_ts":  None,})

# ────────────────────────── WEBSOCKET HANDLERS ─────────────────────
async def websocket_manager(url: str, handler: callable):
    while True:
        try:
            async with websockets.connect(url, ping_interval=20, ping_timeout=20) as ws:
                async for message in ws:
                    await handler(message)
        except Exception as e:
            logging.warning("Connection closed (%s): %s – reconnecting in %ss", url, e, RECONNECT_DELAY)
            await asyncio.sleep(RECONNECT_DELAY)

FUNDING_INTERVAL_MS = 8 * 60 * 60 * 1000  # 8 hours in milliseconds

def create_handler(label: str, *fields):
    async def handler(message):
        try:
            msg     = json.loads(message)
            symbol  = msg["stream"].split("@")[0]
            data    = msg["data"]

            for field, key in fields:
                val = data[key]
                market_data[symbol][field] = float(val) if isinstance(val, str) else val

            # Add time_to_funding_min for perp_mark data
            if label == "perp_mark" and "T" in data:
                now_ms = int(time.time() * 1000)
                time_since_last_funding = now_ms % FUNDING_INTERVAL_MS
                time_to_next_funding_ms = FUNDING_INTERVAL_MS - time_since_last_funding
                market_data[symbol]["time_to_funding_min"] = time_to_next_funding_ms / 60000  # in minutes

        except (KeyError, ValueError, TypeError) as e:
            logging.error("Processing error: %s – message: %s", e, message)
    return handler

spot_handler = create_handler(
    "spot",
    ("spot_bid", "b"), ("spot_ask", "a")
)
perp_book_handler = create_handler(
    "perp_book",
    ("perp_bid", "b"), ("perp_ask", "a")
)
perp_mark_handler = create_handler(
    "perp_mark",
    ("funding", "i"), ("mark_ts", "T")
)



### Tick Sizes

In [197]:
def get_tick_sizes():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()

    tick_sizes = {}

    for symbol_info in data['symbols']:
        symbol = symbol_info['symbol']
        for f in symbol_info['filters']:
            if f['filterType'] == 'PRICE_FILTER':
                tick_sizes[symbol] = float(f['tickSize'])
                break

    return tick_sizes

# Example usage:
tick_size_dict = get_tick_sizes()


In [198]:
def get_symbol_filters():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()

    symbol_filters = {}

    for symbol_info in data['symbols']:
        symbol = symbol_info['symbol']
        filters = {}
        for f in symbol_info['filters']:
            if f['filterType'] == 'PRICE_FILTER':
                filters['tickSize'] = float(f['tickSize'])
            elif f['filterType'] == 'LOT_SIZE':
                filters['stepSize'] = float(f['stepSize'])
                filters['minQty'] = float(f['minQty'])
        if filters:
            symbol_filters[symbol] = filters

    return symbol_filters

### Execute Trades

In [199]:
class FuturesTrader:
    def __init__(self, api_key, api_secret):
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        logging.basicConfig(level=logging.INFO)

        # Connect to Binance Futures Testnet
        self.client = Client(api_key, api_secret)
        self.client.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'

        try:
            account = self.client.futures_account()
            self.logger.info("✅ Connected to Binance Futures Testnet")
        except BinanceAPIException as e:
            self.logger.error(f"❌ Connection failed: {e}")
            raise

    def place_market_order(self, symbol, side, quantity):
        try:
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type=FUTURE_ORDER_TYPE_MARKET,
                quantity=quantity
            )
            self.logger.info(f"✅ Market order placed: {order}")
            return order
        except BinanceAPIException as e:
            self.logger.error(f"❌ Market order failed: {e}")
            return None

    def place_limit_order(self, symbol, side, quantity, price):
        try:
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type=FUTURE_ORDER_TYPE_LIMIT,
                timeInForce=TIME_IN_FORCE_IOC,
                quantity=quantity,
                price=price
            )
            self.logger.info(f"✅ Limit order placed: {order}")
            return order
        except BinanceAPIException as e:
            self.logger.error(f"❌ Limit order failed: {e}")
            return None


    def get_position_pnls(self):
        try:
            positions = self.client.futures_position_information()
            results = []
    
            for p in positions:
                position_amt = float(p['positionAmt'])
                if position_amt == 0.0:
                    continue  # skip empty positions
    
                symbol = p['symbol']
                entry_price = float(p['entryPrice'])
                mark_price = float(p['markPrice'])
                unrealized_pnl = float(p['unRealizedProfit'])
    
                direction = "LONG" if position_amt > 0 else "SHORT"
    
                results.append({
                    "symbol": symbol,
                    "position": position_amt,
                    "direction": direction,
                    "entry_price": entry_price,
                    "mark_price": mark_price,
                    "unrealized_pnl": unrealized_pnl
                })
    
            return results
    
        except Exception as e:
            logging.error(f"Failed to fetch position PnLs: {e}")
            return []


In [200]:
class SpotTrader:
    def __init__(self, api_key_spot, api_secret_spot):
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        logging.basicConfig(level=logging.INFO)

        self.client = Client(api_key_spot, api_secret_spot)
        self.client.API_URL = 'https://testnet.binance.vision/api'

        try:
            account = self.client.get_account()
            self.logger.info("✅ Connected to Binance Spot Testnet")
        except BinanceAPIException as e:
            self.logger.error(f"❌ Connection failed: {e}")
            raise

    def place_market_order(self, symbol, side, quantity):
        try:
            order = self.client.create_order(
                symbol=symbol,
                side=side,
                type='MARKET',
                quantity=quantity
            )
            self.logger.info(f"✅ Market order placed: {order}")
            return order
        except BinanceAPIException as e:
            self.logger.error(f"❌ Market order failed: {e}")
            return None

    def place_limit_order(self, symbol, side, quantity, price):
        try:
            order = self.client.create_order(
                symbol=symbol,
                side=side,
                type='LIMIT',
                timeInForce='IOC',
                quantity=quantity,
                price=price
            )
            self.logger.info(f"✅ Limit order placed: {order}")
            return order
        except BinanceAPIException as e:
            self.logger.error(f"❌ Limit order failed: {e}")
            return None

    # def get_balances(self):
    #     try:
    #         account = self.client.get_account()
    #         balances = account['balances']
    #         non_zero = [
    #             {"asset": b["asset"], "free": b["free"], "locked": b["locked"]}
    #             for b in balances if float(b["free"]) > 0 or float(b["locked"]) > 0
    #         ]
    #         return non_zero
    #     except BinanceAPIException as e:
    #         self.logger.error(f"❌ Failed to fetch balances: {e}")
    #         return []


In [201]:
futures_trader = FuturesTrader(api_key, api_secret)
spot_trader = SpotTrader(api_key_spot, api_secret_spot)

2025-06-27 19:55:59,596 [INFO ] ✅ Connected to Binance Futures Testnet
2025-06-27 19:55:59,813 [INFO ] ✅ Connected to Binance Spot Testnet


### Streaming Data

In [202]:
trade_log = []

SPOT_FEE_RATE = 0.001    # 0.1%
FUTURES_FEE_RATE = 0.0005  # 0.05%
EDGE_THRESHOLD = 0.001
quantity = 5
tick_sizes = get_tick_sizes()

# ledger = TradeLedger()

In [203]:
async def display_top_edges():
    ranked = []
    for symbol, d in market_data.items():
        try:
            if None in (d["spot_ask"], d["perp_bid"], d["spot_bid"], d["perp_ask"]):
                continue

            # NEW: calculate % spreads and filter out wide bid/asks
            spot_spread_pct = abs(d["spot_ask"] - d["spot_bid"]) / ((d["spot_ask"] + d["spot_bid"]) / 2)
            perp_spread_pct = abs(d["perp_ask"] - d["perp_bid"]) / ((d["perp_ask"] + d["perp_bid"]) / 2)

            if spot_spread_pct > 0.005 or perp_spread_pct > 0.005:
                logging.debug(f"[SKIP] {symbol}: Wide spread (Spot: {spot_spread_pct:.2%}, Perp: {perp_spread_pct:.2%})")
                continue
            # END NEW

            spot_mid = (
                (d["spot_bid"] + d["spot_ask"]) / 2
                if d["spot_bid"] and d["spot_ask"] else
                d["spot_bid"] or d["spot_ask"]
            )
            perp_mid = (
                (d["perp_bid"] + d["perp_ask"]) / 2
                if d["perp_bid"] and d["perp_ask"] else
                d["perp_bid"] or d["perp_ask"]
            )

            perp_bid_net = d["perp_bid"] * (1 - FUTURES_FEE_RATE)
            perp_ask_net = d["perp_ask"] * (1 + FUTURES_FEE_RATE)
            spot_bid_net = d["spot_bid"] * (1 - SPOT_FEE_RATE)
            spot_ask_net = d["spot_ask"] * (1 + SPOT_FEE_RATE)

            edge = (
                (perp_bid_net - spot_ask_net)/ spot_ask_net
                if abs(perp_bid_net - spot_ask_net) > abs(perp_ask_net - spot_bid_net)
                else (perp_ask_net - spot_bid_net)/ spot_bid_net
            ) 
            
            funding = d["funding"] or 0.0
            time_to_funding_min = d.get("time_to_funding_min")

            ranked.append((
                symbol.upper(),  # symbol
                edge,            # edge
                spot_mid,
                perp_mid,
                funding,
                time_to_funding_min,
                d["perp_bid"],
                d["perp_ask"],
                d["spot_ask"],
                d["spot_bid"]
            ))

        except Exception as e:
            logging.debug("Error processing %s: %s", symbol, e)

    ranked.sort(key=lambda x: x[1], reverse=True)

    if ranked:
        sym, edge, *_rest, perp_bid, perp_ask, spot_ask, spot_bid = ranked[0]
        funding = _rest[2]               # index 2 = funding
        time_to_funding_min = _rest[3]  # index 3 = time to funding

        time_str = f"{time_to_funding_min:.2f} min" if time_to_funding_min is not None else "N/A"
        logging.info(f" BEST EDGE: {sym} | Edge:{edge:+.6f} | Funding: {funding:+.6f} | Time to funding: {time_str}")
        return sym, edge, perp_bid, perp_ask, spot_ask, spot_bid, funding, time_to_funding_min

    return None, None, None, None, None, None, None, None


async def start_websockets():
    await asyncio.gather(
        websocket_manager(stream_urls["spot"],      spot_handler),
        websocket_manager(stream_urls["perp_book"], perp_book_handler),
        websocket_manager(stream_urls["perp_mark"], perp_mark_handler),
    )

async def main():
    ws_task = asyncio.create_task(start_websockets())

    await asyncio.sleep(1)

    try:
        while True:
            await display_top_edges()
            await asyncio.sleep(1)
    except KeyboardInterrupt:
        logging.info("Shutting down gracefully…")
        ws_task.cancel()
        await asyncio.gather(ws_task, return_exceptions=True)

## Accounts Futures and Spot

In [204]:
class BinanceFuturesAccount:
    def __init__(self, api_key, api_secret, testnet=True):
        base_url = "https://testnet.binancefuture.com" if testnet else None
        self.client = UMFutures(key=api_key, secret=api_secret, base_url=base_url)

    def get_wallet_info(self, asset="USDT"):
        try:
            # Get balances
            account_info = self.client.account()
            balance = next((b for b in account_info['assets'] if b['asset'] == asset), None)
            if not balance:
                raise ValueError(f"Asset {asset} not found in account.")

            wallet_balance = float(balance['walletBalance'])
            available_balance = float(balance['availableBalance'])
            margin_balance = float(balance['marginBalance'])

            # Get unrealized PnL from positions (some may not have marginType)
            position_info = self.client.get_position_risk()
            unrealized_pnl = sum(float(p.get('unRealizedProfit', 0.0)) for p in position_info)

            return {
                "wallet_balance": wallet_balance,
                "available_balance": available_balance,
                "margin_balance": margin_balance,
                "unrealized_pnl": unrealized_pnl
            }

        except Exception as e:
            logging.error(f"Failed to fetch balance: {e}")
            return None

    def print_wallet_status(self, asset="USDT"):
        data = self.get_wallet_info(asset)
        if data:
            print(f"\n------ {asset} ACCOUNT STATUS ------")
            print(f"Wallet Balance    : {data['wallet_balance']}")
            print(f"Available Balance : {data['available_balance']}")
            print(f"Margin Balance    : {data['margin_balance']}")
            print(f"Unrealized PnL    : {data['unrealized_pnl']}")
            print("-------------------------------------\n")
        else:
            print("❌ Failed to fetch account data.")

    def get_nav(self):
        """Returns total NAV (wallet balance + unrealized PnL) in USDT."""
        try:
            account_info = self.client.account()
            balance = next((b for b in account_info['assets'] if b['asset'] == "USDT"), None)
            if not balance:
                return 0.0
    
            wallet_balance = float(balance['walletBalance'])
    
            position_info = self.client.get_position_risk()
            unrealized_pnl = sum(float(p.get('unRealizedProfit', 0.0)) for p in position_info)
    
            total_nav = wallet_balance + unrealized_pnl
            return round(total_nav, 2)
        except Exception as e:
            logging.error(f"[FuturesAccount] Failed to fetch NAV: {e}")
            return 0.0


In [205]:
class BinanceSpotAccount:
    def __init__(self, api_key_spot, api_secret_spot, testnet = True):
        self.client = Client(api_key_spot, api_secret_spot)
        self.client.API_URL = 'https://testnet.binance.vision/api'

    def get_wallet_info(self, asset="USDT"):
        try:
            account_info = self.client.get_account()
            balance = next((b for b in account_info['balances'] if b['asset'] == asset), None)
            if not balance:
                raise ValueError(f"Asset {asset} not found in account.")

            return {
                "free": float(balance['free']),
                "locked": float(balance['locked']),
                "total": float(balance['free']) + float(balance['locked']),
            }

        except Exception as e:
            logging.error(f"Failed to fetch spot balance: {e}")
            return None

    def print_wallet_status(self, asset="USDT"):
        data = self.get_wallet_info(asset)
        if data:
            print(f"\n------ SPOT {asset} ACCOUNT STATUS ------")
            print(f"Free Balance   : {data['free']}")
            print(f"Locked Balance : {data['locked']}")
            print(f"Total Balance  : {data['total']}")
            print("-----------------------------------------\n")
        else:
            print("❌ Failed to fetch spot account data.")

    def get_nav(self, price_lookup):
        """Returns total spot NAV in USDT."""
        try:
            account_info = self.client.get_account()
            total_nav = 0.0
            for balance in account_info["balances"]:
                asset = balance["asset"]
                free = float(balance["free"])
                locked = float(balance["locked"])
                total = free + locked
                if total > 0:
                    if asset == "USDT":
                        total_nav += total
                    else:
                        symbol = f"{asset}USDT"
                        price = price_lookup.get(symbol)
                        if price:
                            total_nav += total * price
            return round(total_nav, 2)
        except Exception as e:
            logging.error(f"[SpotAccount] Failed to calculate NAV: {e}")
            return 0.0


### Portfolio Management

In [206]:
class PortfolioManager:
    def __init__(self, initial_capital, max_positions=5):
        self.positions = {}  # {symbol: trade_details}
        self.initial_capital = initial_capital
        self.max_positions = max_positions
        self.max_notional_per_position = initial_capital / max_positions

    def add_position(self, symbol, trade_details):
        self.positions[symbol] = trade_details

    def remove_position(self, symbol):
        if symbol in self.positions:
            del self.positions[symbol]

    def is_in_position(self, symbol):
        return symbol in self.positions

    def get_position(self, symbol):
        return self.positions.get(symbol)

    def get_open_positions(self):
        return self.positions.copy()

    def num_open_positions(self):
        return len(self.positions)

    def can_open_position(self, notional: float):
        if self.num_open_positions() >= self.max_positions:
            return False
        if notional > self.max_notional_per_position:
            return False
        return True

    def mark_to_market_pnl(self, symbol, spot_bid, spot_ask, futures_bid, futures_ask):
        position = self.get_position(symbol)
        if not position:
            return None

        spot_leg = position["spot_leg"]
        futures_leg = position["futures_leg"]

        # Spot PnL
        spot_mkt_price = spot_bid if spot_leg["side"] == "BUY" else spot_ask
        spot_pnl = (spot_mkt_price - spot_leg["price"]) * spot_leg["quantity"]
        if spot_leg["side"] == "SELL":
            spot_pnl *= -1

        # Futures PnL
        futures_mkt_price = futures_bid if futures_leg["side"] == "BUY" else futures_ask
        futures_pnl = (futures_mkt_price - futures_leg["price"]) * futures_leg["quantity"]
        if futures_leg["side"] == "SELL":
            futures_pnl *= -1

        return round(spot_pnl + futures_pnl, 4)


portfolio_manager = PortfolioManager(initial_capital=INITIAL_CAPITAL, max_positions=5)

In [207]:
# portfolio_manager = PortfolioManager()
# portfolio_manager = PortfolioManager(initial_capital=INITIAL_CAPITAL, max_positions=5)

In [208]:
## Closed Positions

class Portfolio:
    def __init__(self):
        self.trade_history = []   # List of all trades
        self.closed_trades = []   # Trades you've manually or automatically closed

    def log_trade(self, trade_details):
        self.trade_history.append(trade_details)

    def close_trade(self, symbol, final_spot_price, final_futures_price):
        # Close the position manually and compute final PnL
        for trade in self.trade_history:
            if trade["symbol"] == symbol and trade not in self.closed_trades:
                spot_leg = trade["spot_leg"]
                futures_leg = trade["futures_leg"]
                qty = spot_leg["quantity"]

                # Spot PnL
                if spot_leg["side"] == "BUY":
                    spot_pnl = (final_spot_price - spot_leg["price"]) * qty
                else:
                    spot_pnl = (spot_leg["price"] - final_spot_price) * qty

                # Futures PnL
                if futures_leg["side"] == "BUY":
                    futures_pnl = (final_futures_price - futures_leg["price"]) * qty
                else:
                    futures_pnl = (futures_leg["price"] - final_futures_price) * qty

                total_pnl = round(spot_pnl + futures_pnl, 4)
                trade["final_pnl"] = total_pnl
                trade["exit_time"] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

                self.closed_trades.append(trade)
                return trade
        return None

    def show_open_positions(self):
        return [t for t in self.trade_history if t not in self.closed_trades]

    def show_closed_positions(self):
        return self.closed_trades

    def total_realized_pnl(self):
        return round(sum(t.get("final_pnl", 0) for t in self.closed_trades), 4)

portfolio = Portfolio()

### Trade and Entry

In [209]:
def adjust_price_to_tick_size(price: float, tick_size: float, method='round') -> float:

    precision = int(round(-math.log10(tick_size)))
    if method == 'floor':
        adjusted = math.floor(price / tick_size) * tick_size
    elif method == 'ceil':
        adjusted = math.ceil(price / tick_size) * tick_size
    else:
        adjusted = round(price / tick_size) * tick_size
    return round(adjusted, precision)

In [210]:
def adjust_price_to_tick_size(price: float, tick_size: float, method='round') -> float:
    precision = int(round(-math.log10(tick_size)))
    if method == 'floor':
        adjusted = math.floor(price / tick_size) * tick_size
    elif method == 'ceil':
        adjusted = math.ceil(price / tick_size) * tick_size
    else:
        adjusted = round(price / tick_size) * tick_size
    return round(adjusted, precision)


def adjust_quantity_to_step_size(quantity: float, step_size: float) -> float:
    precision = int(round(-math.log10(step_size)))
    adjusted = math.floor(quantity / step_size) * step_size
    return round(adjusted, precision)


def adjust_order(symbol: str, price: float, quantity: float, filters: dict, price_method='round') -> dict:
    if symbol not in filters:
        raise ValueError(f"No filter data available for symbol {symbol}")

    tick_size = filters[symbol]['tickSize']
    step_size = filters[symbol]['stepSize']
    min_qty = filters[symbol]['minQty']

    adj_price = adjust_price_to_tick_size(price, tick_size, method=price_method)
    adj_qty = adjust_quantity_to_step_size(quantity, step_size)

    if adj_qty < min_qty:
        raise ValueError(f"Adjusted quantity {adj_qty} is below the minimum allowed {min_qty}")

    return {
        'price': adj_price,
        'quantity': adj_qty
    }


In [211]:
MAX_POSITIONS = 5
POSITION_SIZE = INITIAL_CAPITAL / MAX_POSITIONS

def can_open_new_position(portfolio_manager):
    open_positions = portfolio_manager.get_open_positions()

    if len(open_positions) >= MAX_POSITIONS:
        logging.info("❌ Max position count reached. Cannot open new trade.")
        return False

    current_used_margin = sum(pos.get("allocated_usdt", POSITION_SIZE) for pos in open_positions.values())
    remaining_margin = INITIAL_CAPITAL - current_used_margin

    if remaining_margin < POSITION_SIZE:
        logging.info("❌ Insufficient available margin. Needed: %.2f, Available: %.2f", POSITION_SIZE, remaining_margin)
        return False

    return True

In [212]:
# class LiquidityChecker:
#     def __init__(self, futures_trader, spot_trader):
#         self.futures_trader = futures_trader
#         self.spot_trader = spot_trader
        
#     def check_orderbook_depth(self, symbol, side='both', min_depth_usdt=1000):
#         """Check if there's enough liquidity to execute our trade"""
#         try:
#             # Get futures orderbook
#             futures_symbol = symbol.replace('/', '')
#             futures_book = self.futures_trader.client.futures_order_book(symbol=futures_symbol, limit=10)
            
#             # Get spot orderbook  
#             spot_book = self.spot_trader.client.get_order_book(symbol=futures_symbol, limit=10)
            
#             # Calculate bid/ask depths in USDT
#             futures_bid_depth = sum(float(bid[0]) * float(bid[1]) for bid in futures_book['bids'][:5])
#             futures_ask_depth = sum(float(ask[0]) * float(ask[1]) for ask in futures_book['asks'][:5])
            
#             spot_bid_depth = sum(float(bid[0]) * float(bid[1]) for bid in spot_book['bids'][:5])
#             spot_ask_depth = sum(float(ask[0]) * float(ask[1]) for ask in spot_book['asks'][:5])
            
#             liquidity_info = {
#                 'futures_bid_depth': futures_bid_depth,
#                 'futures_ask_depth': futures_ask_depth,
#                 'spot_bid_depth': spot_bid_depth,
#                 'spot_ask_depth': spot_ask_depth,
#                 'sufficient_liquidity': True
#             }
            
#             # Check if we have enough liquidity for our trade size
#             if side in ['short_perp', 'both']:
#                 if futures_bid_depth < min_depth_usdt:
#                     liquidity_info['sufficient_liquidity'] = False
#                     liquidity_info['reason'] = f"Futures bid depth too low: ${futures_bid_depth:.0f}"
                    
#             if side in ['long_spot', 'both']:
#                 if spot_ask_depth < min_depth_usdt:
#                     liquidity_info['sufficient_liquidity'] = False
#                     liquidity_info['reason'] = f"Spot ask depth too low: ${spot_ask_depth:.0f}"
            
#             return liquidity_info
            
#         except Exception as e:
#             logging.error(f"Error checking liquidity for {symbol}: {e}")
#             return {'sufficient_liquidity': False, 'reason': f"API error: {e}"}
    
#     def check_spread_quality(self, symbol, max_spread_pct=0.5):
#         """Check if bid-ask spreads are reasonable"""
#         try:
#             futures_symbol = symbol.replace('/', '')
            
#             # Get best bid/ask prices
#             futures_ticker = self.futures_trader.client.futures_symbol_ticker(symbol=futures_symbol)
#             spot_ticker = self.spot_trader.client.get_symbol_ticker(symbol=futures_symbol)
            
#             futures_book = self.futures_trader.client.futures_order_book(symbol=futures_symbol, limit=1)
#             spot_book = self.spot_trader.client.get_order_book(symbol=futures_symbol, limit=1)
            
#             futures_bid = float(futures_book['bids'][0][0])
#             futures_ask = float(futures_book['asks'][0][0])
#             spot_bid = float(spot_book['bids'][0][0])
#             spot_ask = float(spot_book['asks'][0][0])
            
#             futures_spread_pct = (futures_ask - futures_bid) / futures_bid * 100
#             spot_spread_pct = (spot_ask - spot_bid) / spot_bid * 100
            
#             spread_info = {
#                 'futures_spread_pct': futures_spread_pct,
#                 'spot_spread_pct': spot_spread_pct,
#                 'good_spreads': futures_spread_pct < max_spread_pct and spot_spread_pct < max_spread_pct
#             }
            
#             if not spread_info['good_spreads']:
#                 spread_info['reason'] = f"Wide spreads: Futures {futures_spread_pct:.2f}%, Spot {spot_spread_pct:.2f}%"
            
#             return spread_info
            
#         except Exception as e:
#             return {'good_spreads': False, 'reason': f"Spread check error: {e}"}

In [213]:
async def trade_top_opportunity(futures_trader, spot_trader):
    sym, edge, perp_bid, perp_ask, spot_ask, spot_bid, funding, time_to_funding_min = await display_top_edges()
    if sym is None or edge is None:
        return

    symbol_key = sym.replace("/", "")  # e.g., BTC/USDT → BTCUSDT

    if not can_open_new_position(portfolio_manager):
        logging.info("⚠️ Trade skipped: position limits or capital constraints.")
        return

    if portfolio_manager.is_in_position(symbol_key):
        logging.info(f"⛔ Trade skipped: already in position for {symbol_key}")
        return


    # Check spread condition
    mid_perp = (perp_bid + perp_ask) / 2
    mid_spot = (spot_bid + spot_ask) / 2
    spread_pct = abs(mid_perp - mid_spot) / mid_spot
    if spread_pct > 0.005:  # 0.5%
        logging.warning(f"⚠️ Skipping trade: Bid-ask spread > 0.5% ({spread_pct:.4%})")
        return

    logging.info("------------------------------- Trade -----------------------------------")

    futures_symbol = sym.replace("/", "")
    spot_symbol = sym.upper()
    tick_size = tick_sizes.get(sym)

    # Dynamically calculate quantity based on POSITION_SIZE
    # quantity = round(POSITION_SIZE / mid_perp, 4)
    # if tick_size:
    #     quantity = round((quantity // tick_size) * tick_size, 4)

    symbol_filters = get_symbol_filters()  # Cache this externally in prod
    # filters = symbol_filters.get(symbol_key)
    # raw_quantity = POSITION_SIZE / mid_perp
    # adjusted = adjust_order(symbol_key, mid_perp, raw_quantity, filters)

    filters = symbol_filters.get(symbol_key)
    if filters is None:
        logging.warning(f"⚠️ No Binance filters found for {symbol_key}, skipping trade.")
        return

    filters = symbol_filters.get(symbol_key)
    if filters is None:
        logging.warning(f"⚠️ No Binance filters for {symbol_key}, skipping.")
        return
    
    raw_qty = POSITION_SIZE / mid_perp
    adj = adjust_order(symbol_key, mid_perp, raw_qty, symbol_filters)
    price_for_trade = adj["price"]
    quantity = adj["quantity"]

    trade_details = {
        "time": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
        "symbol": futures_symbol,
        "edge": edge,
        "entry": None,
        "tp": None,
        "sl": None,
        "side": None,
        "notional": None,
        "spot_cost": None,
        "futures_cost": None,
        "total_cost": None,
        "allocated_usdt": None,
        "spot_leg": {
            "side": None,
            "price": None,
            "quantity": quantity
        },
        "futures_leg": {
            "side": None,
            "price": None,
            "quantity": quantity
        }
    }

    def short_perp_long_spot():
        raw_entry_price = perp_bid
        # entry_price = round(raw_entry_price * (1 - FUTURES_FEE_RATE), 2)
        raw_spot_price = spot_ask
        # spot_entry_price = round(raw_spot_price * (1 + SPOT_FEE_RATE), 2)
        entry_price_raw = raw_entry_price * (1 - FUTURES_FEE_RATE)
        spot_entry_price_raw = raw_spot_price * (1 + SPOT_FEE_RATE)
        
        entry_price = adjust_price_to_tick_size(entry_price_raw, filters["tickSize"], "floor")
        spot_entry_price = adjust_price_to_tick_size(spot_entry_price_raw, filters["tickSize"], "ceil")

        futures_trader.place_limit_order(futures_symbol, SIDE_SELL, quantity, entry_price)
        spot_trader.place_limit_order(spot_symbol, SIDE_BUY, quantity, spot_entry_price)

        notional = quantity * entry_price
        trade_details.update({
            "entry": entry_price,
            "side": "SELL (perp) / BUY (spot)",
            "notional": notional,
            "allocated_usdt": notional,
            "spot_cost": round(quantity * raw_spot_price * SPOT_FEE_RATE, 4),
            "futures_cost": round(quantity * raw_entry_price * FUTURES_FEE_RATE, 4),
            "spot_leg": {
                "side": "BUY",
                "price": spot_entry_price,
                "quantity": quantity
            },
            "futures_leg": {
                "side": "SELL",
                "price": entry_price,
                "quantity": quantity
            }
        })

    def long_perp_short_spot():
        raw_entry_price = perp_ask
        # entry_price = round(raw_entry_price * (1 + FUTURES_FEE_RATE), 2)
        raw_spot_price = spot_bid
        # spot_entry_price = round(raw_spot_price * (1 - SPOT_FEE_RATE), 2)

        entry_price_raw = raw_entry_price * (1 + FUTURES_FEE_RATE)
        spot_entry_price_raw = raw_spot_price * (1 - SPOT_FEE_RATE)
        
        entry_price = adjust_price_to_tick_size(entry_price_raw, filters["tickSize"], "ceil")
        spot_entry_price = adjust_price_to_tick_size(spot_entry_price_raw, filters["tickSize"], "floor")

        futures_trader.place_limit_order(futures_symbol, SIDE_BUY, quantity, entry_price)
        spot_trader.place_limit_order(spot_symbol, SIDE_SELL, quantity, spot_entry_price)

        notional = quantity * entry_price
        trade_details.update({
            "entry": entry_price,
            "side": "BUY (perp) / SELL (spot)",
            "notional": notional,
            "allocated_usdt": notional,
            "spot_cost": round(quantity * raw_spot_price * SPOT_FEE_RATE, 4),
            "futures_cost": round(quantity * raw_entry_price * FUTURES_FEE_RATE, 4),
            "spot_leg": {
                "side": "SELL",
                "price": spot_entry_price,
                "quantity": quantity
            },
            "futures_leg": {
                "side": "BUY",
                "price": entry_price,
                "quantity": quantity
            }
        })

    # --- Entry logic ---
    if edge > EDGE_THRESHOLD and funding > 0:
        logging.info("✅ Executing main condition: Short perp, long spot.")
        short_perp_long_spot()

    elif edge < -EDGE_THRESHOLD and funding < 0:
        logging.info("✅ Executing main condition: Long perp, short spot.")
        long_perp_short_spot()

    elif edge > EDGE_THRESHOLD and funding < 0 and time_to_funding_min and time_to_funding_min > 360:
        if perp_bid > spot_ask:
            logging.info("🔄 Executing anticipatory trade: Short perp, long spot (expect funding reversal).")
            short_perp_long_spot()
        else:
            logging.info("⚠️ Skipping anticipatory trade: perp_bid not > spot_ask.")

    elif edge < -EDGE_THRESHOLD and funding > 0 and time_to_funding_min and time_to_funding_min > 360:
        if perp_ask < spot_bid:
            logging.info("🔄 Executing anticipatory trade: Long perp, short spot (expect funding reversal).")
            long_perp_short_spot()
        else:
            logging.info("⚠️ Skipping anticipatory trade: perp_ask not < spot_bid.")

    else:
        logging.info("⚠️ Edge or funding condition not met — no trade executed.")
        return

    # --- Final Trade Logging ---
    trade_details["total_cost"] = round(trade_details["spot_cost"] + trade_details["futures_cost"], 4) * 2
    mtm_pnl = portfolio_manager.mark_to_market_pnl(sym, spot_bid, spot_ask, perp_bid, perp_ask)
    trade_details["mtm_pnl"] = mtm_pnl

    if portfolio_manager.is_in_position(symbol_key) == True:
        trade_log.append(trade_details)
        portfolio.log_trade(trade_details)
        portfolio_manager.add_position(sym, trade_details)
        logging.info(f"✅ Logged trade: {trade_details}")
    else:
        print("Limit order submitted but not in book")
    
    # trade_log.append(trade_details)
    # portfolio.log_trade(trade_details)
    # portfolio_manager.add_position(sym, trade_details)
    # logging.info(f"✅ Logged trade: {trade_details}")


In [214]:
async def check_exit_conditions():
    open_positions = portfolio_manager.get_open_positions()
    required_fields = ["spot_bid", "spot_ask", "perp_bid", "perp_ask", "funding", "time_to_funding_min"]

    for sym, trade in open_positions.items():
        futures_symbol = sym.replace("/", "").lower()
        spot_symbol = sym.replace("/", "").lower()

        # Check for missing market data
        missing_fields = [field for field in required_fields if market_data[futures_symbol].get(field) is None]
        if missing_fields:
            logging.warning(f"⚠️ Incomplete market data for {sym}: missing {missing_fields} - skipping exit check")
            continue

        try:
            perp_bid = market_data[futures_symbol]["perp_bid"]
            perp_ask = market_data[futures_symbol]["perp_ask"]
            spot_bid = market_data[spot_symbol]["spot_bid"]
            spot_ask = market_data[spot_symbol]["spot_ask"]
            funding = market_data[futures_symbol]["funding"]
            time_to_funding_min = market_data[futures_symbol]["time_to_funding_min"]
        except KeyError:
            logging.warning(f"❌ Market data not available for {sym}")
            continue

        # Calculate current edge
        current_edge = (perp_bid + perp_ask) / 2 - (spot_bid + spot_ask) / 2
        entry_edge = trade["edge"]

        # Condition 1: edge has reverted (crossed zero)
        edge_condition_met = (
            (entry_edge > 0 and current_edge <= 0) or
            (entry_edge < 0 and current_edge >= 0)
        )

        # Condition 2: funding collected after funding time has passed
        funding_collected = trade.get("funding_collected", False)
        if not funding_collected:
            if time_to_funding_min is not None and time_to_funding_min <= 0:
                if (entry_edge > 0 and funding > 0) or (entry_edge < 0 and funding < 0):
                    trade["funding_collected"] = True
                    funding_collected = True
                    logging.info(f"✅ Funding collected for {sym}")

        # --- Exit if both conditions are met ---
        if edge_condition_met and funding_collected:
            logging.info(f"📤 Exiting trade: {sym} | Reason: edge reverted AND funding collected")

            try:
                if "SELL" in trade["side"]:
                    futures_trader.place_market_order(futures_symbol.upper(), SIDE_BUY, trade["futures_leg"]["quantity"])
                    spot_trader.place_market_order(spot_symbol.upper(), SIDE_SELL, trade["spot_leg"]["quantity"])
                else:
                    futures_trader.place_market_order(futures_symbol.upper(), SIDE_SELL, trade["futures_leg"]["quantity"])
                    spot_trader.place_market_order(spot_symbol.upper(), SIDE_BUY, trade["spot_leg"]["quantity"])
            except Exception as e:
                logging.error(f"❌ Market order failed for {sym}: {e}")
                continue

            portfolio_manager.remove_position(sym)
            logging.info(f"✅ Position closed for {sym}")


In [215]:
# ###Updates the trade_log DataFrame with current position status and mark-to-market PnL.

# def sync_position_status(trade_log_df, pnl_snapshot):
#     pnl_map = {p['symbol'].upper(): p for p in pnl_snapshot}

#     statuses = []
#     mtm_pnls = []

#     for _, row in trade_log_df.iterrows():
#         symbol = row['symbol'].upper()
#         pnl_entry = pnl_map.get(symbol)

#         if pnl_entry:
#             position_amt = float(pnl_entry['position'])
#             unrealized_pnl = float(pnl_entry['unrealized_pnl'])
#             statuses.append("OPEN" if position_amt != 0 else "CLOSED")
#             mtm_pnls.append(unrealized_pnl)
#         else:
#             statuses.append("NO_POSITION")
#             mtm_pnls.append(None)

#     trade_log_df["status"] = statuses
#     trade_log_df["mtm_pnl"] = mtm_pnls
#     return trade_log_df


In [216]:
nav_history = []

In [217]:
async def log_nav(spot_account, futures_account):
    try:
        # Get real-time spot prices
        tickers = spot_account.client.get_all_tickers()
        price_lookup = {item["symbol"]: float(item["price"]) for item in tickers}

        # Get NAVs
        spot_nav = spot_account.get_nav(price_lookup)
        futures_nav = futures_account.get_nav()  # Already returns float total

        total_nav = spot_nav + futures_nav

        logging.info(f"[NAV] Spot NAV: {spot_nav:.2f} | Futures NAV: {futures_nav:.2f} | Total NAV: {total_nav:.2f}")

    except Exception as e:
        logging.error(f"[NAV ERROR] Failed to log NAV: {e}")


In [218]:
def plot_nav_history():
    if not nav_history:
        logging.warning("No NAV history available to plot.")
        return

    df = pd.DataFrame(nav_history)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["time"], y=df["spot_balance"],
                             mode='lines+markers', name='Spot Balance'))
    fig.add_trace(go.Scatter(x=df["time"], y=df["futures_balance"],
                             mode='lines+markers', name='Futures Balance'))
    fig.add_trace(go.Scatter(x=df["time"], y=df["total_nav"],
                             mode='lines+markers', name='Total NAV', line=dict(width=4)))

    fig.update_layout(
        title="Net Asset Value Over Time",
        xaxis_title="Time",
        yaxis_title="USDT",
        template="plotly_dark",
        legend=dict(x=0, y=1.1, orientation="h")
    )

    fig.show()

In [219]:
async def main():
    ws_task = asyncio.create_task(start_websockets())
    await asyncio.sleep(3)

    futures_account = BinanceFuturesAccount(api_key, api_secret, testnet=True)
    spot_account = BinanceSpotAccount(api_key_spot, api_secret_spot, testnet=True)

    try:
        while True:
            await trade_top_opportunity(futures_trader, spot_trader)
            await check_exit_conditions()

            futures_account.print_wallet_status("USDT")
            spot_account.print_wallet_status("USDT")

            await log_nav(spot_account, futures_account)  # Now uses real-time prices

            await asyncio.sleep(5)
    except KeyboardInterrupt:
        logging.info("Shutting down gracefully…")
        ws_task.cancel()
        await asyncio.gather(ws_task, return_exceptions=True)

### Version 2

In [220]:
# async def main():
#     ws_task = asyncio.create_task(start_websockets())
#     await asyncio.sleep(3)

#     futures_account = BinanceFuturesAccount(api_key, api_secret, testnet=True)
#     spot_account = BinanceSpotAccount(api_key_spot, api_secret_spot, testnet=True)

#     try:
#         while True:
#             await trade_top_opportunity_dynamic(futures_trader, spot_trader, max_attempts=5)
#             await check_exit_conditions()

#             futures_account.print_wallet_status("USDT")
#             spot_account.print_wallet_status("USDT")

#             await log_nav(spot_account, futures_account)  # Now uses real-time prices

#             await asyncio.sleep(5)
#     except KeyboardInterrupt:
#         logging.info("Shutting down gracefully…")
#         ws_task.cancel()
#         await asyncio.gather(ws_task, return_exceptions=True)

In [221]:
# Add this to your script to completely reset
def reset_portfolio_data():
    global trade_log
    
    # Clear trade log
    trade_log.clear()
    
    # Reset portfolio manager positions
    portfolio_manager.positions.clear()
    
    # Reset any other tracking data structures
    if hasattr(portfolio, 'trades'):
        portfolio.trades.clear()
    
    print("✅ Portfolio data reset complete")
    print(f"Trade log length: {len(trade_log)}")
    print(f"Portfolio positions: {len(portfolio_manager.positions)}")

# Run this once
reset_portfolio_data()

✅ Portfolio data reset complete
Trade log length: 0
Portfolio positions: 0


In [235]:
if __name__ == "__main__":
    # asyncio.run(main())
    await main()

2025-06-27 20:03:38,540 [INFO ]  BEST EDGE: SCRUSDT | Edge:+0.004840 | Funding: +0.240859 | Time to funding: 236.40 min
2025-06-27 20:03:38,541 [INFO ] ------------------------------- Trade -----------------------------------
/var/folders/y9/3ntrh6355w3466706_tllyx80000gn/T/ipykernel_13382/4108159731.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "time": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
2025-06-27 20:03:38,897 [INFO ] ✅ Executing main condition: Short perp, long spot.
2025-06-27 20:03:39,012 [ERROR] ❌ Limit order failed: APIError(code=-1111): Precision is over the maximum defined for this asset.
2025-06-27 20:03:39,128 [INFO ] ✅ Limit order placed: {'symbol': 'SCRUSDT', 'orderId': 224803, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJdd9db451e7d0a9d6006316', 'transactTime': 1751025819093, 'price': '0.242000

Limit order submitted but not in book

------ USDT ACCOUNT STATUS ------
Wallet Balance    : 14997.66998762
Available Balance : 15017.19355272
Margin Balance    : 15322.37848625
Unrealized PnL    : 324.70849863
-------------------------------------


------ SPOT USDT ACCOUNT STATUS ------
Free Balance   : 268474.55461374
Locked Balance : 0.0
Total Balance  : 268474.55461374
-----------------------------------------



2025-06-27 20:03:39,782 [INFO ] [NAV] Spot NAV: 424580.74 | Futures NAV: 15322.38 | Total NAV: 439903.12


CancelledError: 

In [223]:
pd.DataFrame(trade_log)

""


### Account Money/ Positions

In [224]:
futures_account = BinanceFuturesAccount(api_key, api_secret, testnet=True)
futures_account.print_wallet_status("USDT")


------ USDT ACCOUNT STATUS ------
Wallet Balance    : 14997.66998762
Available Balance : 15016.52784049
Margin Balance    : 15309.06424148
Unrealized PnL    : 311.39425386
-------------------------------------



In [225]:
spot_account = BinanceSpotAccount(api_key_spot, api_secret_spot, testnet=True)
spot_account.print_wallet_status("USDT")


------ SPOT USDT ACCOUNT STATUS ------
Free Balance   : 271476.06501374
Locked Balance : 0.0
Total Balance  : 271476.06501374
-----------------------------------------



In [226]:
pnls = futures_trader.get_position_pnls()

for p in pnls:
    print(p)

{'symbol': 'LOKAUSDT', 'position': -68000.0, 'direction': 'SHORT', 'entry_price': 0.055573125, 'mark_price': 0.05169841, 'unrealized_pnl': 263.48028}
{'symbol': 'OPUSDT', 'position': -1988.3, 'direction': 'SHORT', 'entry_price': 0.5538979681135, 'mark_price': 0.5298, 'unrealized_pnl': 47.91397386}


### Close Open Orders

In [227]:
# # Cancel all open futures orders
# print("=== CANCELING ALL FUTURES ORDERS ===")
# try:
#     open_orders = futures_trader.client.futures_get_open_orders()
#     print(f"Found {len(open_orders)} open futures orders")
    
#     for order in open_orders:
#         print(f"Canceling {order['symbol']} order {order['orderId']}")
#         futures_trader.client.futures_cancel_order(
#             symbol=order['symbol'], 
#             orderId=order['orderId']
#         )
#     print("✅ All futures orders canceled")
# except Exception as e:
#     print(f"Error canceling futures orders: {e}")

# # Cancel all open spot orders
# print("\n=== CANCELING ALL SPOT ORDERS ===")
# try:
#     open_orders = spot_trader.client.get_open_orders()
#     print(f"Found {len(open_orders)} open spot orders")
    
#     for order in open_orders:
#         print(f"Canceling {order['symbol']} order {order['orderId']}")
#         spot_trader.client.cancel_order(
#             symbol=order['symbol'], 
#             orderId=order['orderId']
#         )
#     print("✅ All spot orders canceled")
# except Exception as e:
#     print(f"Error canceling spot orders: {e}")

### Close Actual Positions

In [228]:
# # Close all futures positions
# print("\n=== CLOSING ALL FUTURES POSITIONS ===")
# try:
#     account = futures_trader.client.futures_account()
#     positions = account['positions']
    
#     active_positions = [p for p in positions if float(p['positionAmt']) != 0]
#     print(f"Found {len(active_positions)} active positions")
    
#     for pos in active_positions:
#         symbol = pos['symbol']
#         position_amt = float(pos['positionAmt'])
        
#         if position_amt > 0:  # Long position - sell to close
#             print(f"Closing LONG {symbol}: selling {position_amt}")
#             futures_trader.client.futures_create_order(
#                 symbol=symbol,
#                 side='SELL',
#                 type='MARKET',
#                 quantity=abs(position_amt)
#             )
#         elif position_amt < 0:  # Short position - buy to close
#             print(f"Closing SHORT {symbol}: buying {abs(position_amt)}")
#             futures_trader.client.futures_create_order(
#                 symbol=symbol,
#                 side='BUY',
#                 type='MARKET',
#                 quantity=abs(position_amt)
#             )
    
#     print("✅ All futures positions closed")
# except Exception as e:
#     print(f"Error closing futures positions: {e}")

# # Check spot balances and sell non-USDT assets
# print("\n=== CLOSING SPOT POSITIONS ===")
# try:
#     account = spot_trader.client.get_account()
#     balances = account['balances']
    
#     # Find non-USDT assets with balance > 0
#     crypto_balances = [b for b in balances if float(b['free']) > 0 and b['asset'] != 'USDT']
    
#     for balance in crypto_balances:
#         asset = balance['asset']
#         amount = float(balance['free'])
        
#         if amount > 0:
#             symbol = f"{asset}USDT"
#             print(f"Selling {amount} {asset}")
#             try:
#                 spot_trader.client.create_order(
#                     symbol=symbol,
#                     side='SELL',
#                     type='MARKET',
#                     quantity=amount
#                 )
#             except Exception as e:
#                 print(f"  Could not sell {asset}: {e}")
    
#     print("✅ All spot positions closed")
# except Exception as e:
#     print(f"Error closing spot positions: {e}")

### Clean Slate

In [229]:
# def clean_slate():
#     """Close everything and start fresh"""
#     print("🧹 CLEANING SLATE - CLOSING ALL TRADES...")
    
#     # Cancel orders
#     try:
#         # Futures orders
#         futures_orders = futures_trader.client.futures_get_open_orders()
#         for order in futures_orders:
#             futures_trader.client.futures_cancel_order(symbol=order['symbol'], orderId=order['orderId'])
        
#         # Spot orders  
#         spot_orders = spot_trader.client.get_open_orders()
#         for order in spot_orders:
#             spot_trader.client.cancel_order(symbol=order['symbol'], orderId=order['orderId'])
            
#         print(f"✅ Canceled {len(futures_orders)} futures + {len(spot_orders)} spot orders")
#     except Exception as e:
#         print(f"Error canceling orders: {e}")
    
#     # Close positions
#     try:
#         # Close futures positions
#         account = futures_trader.client.futures_account()
#         for pos in account['positions']:
#             amt = float(pos['positionAmt'])
#             if amt != 0:
#                 side = 'SELL' if amt > 0 else 'BUY'
#                 futures_trader.client.futures_create_order(
#                     symbol=pos['symbol'], side=side, type='MARKET', quantity=abs(amt)
#                 )
#         print("✅ Closed all futures positions")
#     except Exception as e:
#         print(f"Error closing positions: {e}")
    
#     print("🎯 Clean slate complete!")

# # Run it:
# clean_slate()

In [230]:
for p in pnls:
    print(p)

{'symbol': 'LOKAUSDT', 'position': -68000.0, 'direction': 'SHORT', 'entry_price': 0.055573125, 'mark_price': 0.05169841, 'unrealized_pnl': 263.48028}
{'symbol': 'OPUSDT', 'position': -1988.3, 'direction': 'SHORT', 'entry_price': 0.5538979681135, 'mark_price': 0.5298, 'unrealized_pnl': 47.91397386}


In [231]:
client = Client(api_key_spot, api_secret_spot)
client.API_URL = 'https://testnet.binance.vision/api'
account_info = client.get_account()
balances = account_info['balances']

# Print non-zero balances
for asset in balances:
    if float(asset['free']) > 0 or float(asset['locked']) > 0:
        print(f"{asset['asset']}: Free={asset['free']}, Locked={asset['locked']}")
print(account_info)

USDT: Free=271476.06501374, Locked=0.00000000
WIN: Free=18446.00000000, Locked=0.00000000
TRY: Free=18466.00000000, Locked=0.00000000
ZAR: Free=18466.00000000, Locked=0.00000000
UAH: Free=18466.00000000, Locked=0.00000000
DAI: Free=10000.00000000, Locked=0.00000000
BRL: Free=97.00000000, Locked=0.00000000
CELO: Free=34932.60000000, Locked=0.00000000
SHIB: Free=18446.00, Locked=0.00
XEC: Free=18446.00, Locked=0.00
RARE: Free=335042.90000000, Locked=0.00000000
LOKA: Free=231769.40000000, Locked=0.00000000
BTTC: Free=18446.0, Locked=0.0
BSW: Free=1402386.60000000, Locked=0.00000000
OP: Free=5659.84000000, Locked=0.00000000
LUNC: Free=18446.00000000, Locked=0.00000000
PLN: Free=18466.00000000, Locked=0.00000000
RON: Free=18466.00000000, Locked=0.00000000
ARS: Free=18466.00000000, Locked=0.00000000
PEPE: Free=18446.00, Locked=0.00
1000SATS: Free=18446.00, Locked=0.00
BONK: Free=18446.00, Locked=0.00
DYM: Free=148480.90000000, Locked=0.00000000
JPY: Free=18466.00000000, Locked=0.00000000
MXN

In [232]:
BASE_URL = 'https://testnet.binance.vision'

def create_signature(query_string, secret):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def get_spot_positions():
    endpoint = '/api/v3/account'
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp}'
    signature = create_signature(query_string, api_secret_spot)
    headers = {'X-MBX-APIKEY': api_key_spot}
    url = f'{BASE_URL}{endpoint}?{query_string}&signature={signature}'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        non_zero_balances = [bal for bal in data['balances'] if float(bal['free']) > 0 or float(bal['locked']) > 0]
        if not non_zero_balances:
            print("No holdings on spot account.")
        else:
            print("Spot account holdings:")
            for bal in non_zero_balances:
                print(f"Asset: {bal['asset']}, Free: {bal['free']}, Locked: {bal['locked']}")
        return non_zero_balances
    else:
        print(f"Error fetching spot account info: {response.status_code}, {response.text}")
        return None

if __name__ == '__main__':
    get_spot_positions()
    

Spot account holdings:
Asset: USDT, Free: 271476.06501374, Locked: 0.00000000
Asset: WIN, Free: 18446.00000000, Locked: 0.00000000
Asset: TRY, Free: 18466.00000000, Locked: 0.00000000
Asset: ZAR, Free: 18466.00000000, Locked: 0.00000000
Asset: UAH, Free: 18466.00000000, Locked: 0.00000000
Asset: DAI, Free: 10000.00000000, Locked: 0.00000000
Asset: BRL, Free: 97.00000000, Locked: 0.00000000
Asset: CELO, Free: 34932.60000000, Locked: 0.00000000
Asset: SHIB, Free: 18446.00, Locked: 0.00
Asset: XEC, Free: 18446.00, Locked: 0.00
Asset: RARE, Free: 335042.90000000, Locked: 0.00000000
Asset: LOKA, Free: 231769.40000000, Locked: 0.00000000
Asset: BTTC, Free: 18446.0, Locked: 0.0
Asset: BSW, Free: 1402386.60000000, Locked: 0.00000000
Asset: OP, Free: 5659.84000000, Locked: 0.00000000
Asset: LUNC, Free: 18446.00000000, Locked: 0.00000000
Asset: PLN, Free: 18466.00000000, Locked: 0.00000000
Asset: RON, Free: 18466.00000000, Locked: 0.00000000
Asset: ARS, Free: 18466.00000000, Locked: 0.00000000
A

In [233]:
def match_futures_with_spot(futures_trader, spot_trader):
    
    # Get futures positions
    try:
        futures_pnls = futures_trader.get_position_pnls()
    except Exception as e:
        print(f"Error getting futures positions: {e}")
        return None
    
    if not futures_pnls:
        print("No active futures positions found.")
        return None
    
    # Get spot balances
    try:
        account = spot_trader.client.get_account()
        spot_balances = {}
        for balance in account['balances']:
            asset = balance['asset']
            total = float(balance['free']) + float(balance['locked'])
            if total > 0:
                spot_balances[f"{asset}USDT"] = {
                    'asset': asset,
                    'quantity': total,
                    'free': float(balance['free']),
                    'locked': float(balance['locked'])
                }
    except Exception as e:
        print(f"Error getting spot balances: {e}")
        return None
    
    # Create comparison table
    comparison_data = []
    
    for futures_pos in futures_pnls:
        symbol = futures_pos['symbol']
        base_asset = symbol.replace('USDT', '')
        
        # Check if we have corresponding spot position
        spot_holding = spot_balances.get(symbol, {})
        spot_qty = spot_holding.get('quantity', 0)
        
        # Determine hedge status
        futures_amt = float(futures_pos['position'])
        
        if futures_amt > 0 and spot_qty > 0:
            hedge_status = "⚠️ BOTH LONG (No Hedge)"
            hedge_ratio = "N/A"
        elif futures_amt < 0 and spot_qty > 0:
            hedge_status = "✅ HEDGED (Short Perp + Long Spot)"
            hedge_ratio = f"{min(abs(futures_amt), spot_qty) / max(abs(futures_amt), spot_qty) * 100:.1f}%"
        elif futures_amt > 0 and spot_qty == 0:
            hedge_status = "❌ ORPHANED (Long Perp Only)"
            hedge_ratio = "0%"
        elif futures_amt < 0 and spot_qty == 0:
            hedge_status = "❌ ORPHANED (Short Perp Only)"
            hedge_ratio = "0%"
        else:
            hedge_status = "❓ UNKNOWN"
            hedge_ratio = "N/A"
        
        comparison_data.append({
            'Symbol': symbol,
            'Futures Position': futures_amt,
            'Futures Direction': futures_pos['direction'],
            'Spot Quantity': spot_qty,
            'Spot Asset': base_asset if spot_qty > 0 else '-',
            'Hedge Status': hedge_status,
            'Hedge Ratio': hedge_ratio,
            'Futures PnL': float(futures_pos['unrealized_pnl']),
            'Entry Price': float(futures_pos['entry_price']),
            'Mark Price': float(futures_pos['mark_price'])
        })
    
    # Create DataFrame
    df = pd.DataFrame(comparison_data)
    
    # Display the table
    print("=" * 120)
    print("FUTURES POSITIONS vs SPOT HOLDINGS")
    print("=" * 120)
    
    # Format the display
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)
    
    # Show only relevant columns for clean display
    display_df = df[['Symbol', 'Futures Position', 'Futures Direction', 'Spot Quantity', 'Hedge Status', 'Hedge Ratio', 'Futures PnL']].copy()
    
    # Round numbers for cleaner display
    display_df['Futures Position'] = display_df['Futures Position'].round(0).astype(int)
    display_df['Spot Quantity'] = display_df['Spot Quantity'].round(0).astype(int)
    display_df['Futures PnL'] = display_df['Futures PnL'].round(2)
    
    print(display_df.to_string(index=False))
    
    # Summary
    total_pnl = df['Futures PnL'].sum()
    hedged_count = len(df[df['Hedge Status'].str.contains('HEDGED')])
    orphaned_count = len(df[df['Hedge Status'].str.contains('ORPHANED')])
    
    print("\n" + "=" * 60)
    print("SUMMARY")
    print("=" * 60)
    print(f"Total Futures Positions: {len(df)}")
    print(f"Properly Hedged: {hedged_count}")
    print(f"Orphaned (Need Spot): {orphaned_count}")
    print(f"Total Unrealized PnL: ${total_pnl:.2f}")
    
    # Show orphaned positions that need attention
    orphaned = df[df['Hedge Status'].str.contains('ORPHANED')]
    if len(orphaned) > 0:
        print(f"\n🚨 ORPHANED POSITIONS NEEDING ATTENTION:")
        for _, row in orphaned.iterrows():
            action = ""
            if row['Futures Direction'] == 'SHORT':
                action = f"Need to BUY {abs(row['Futures Position'])} {row['Symbol'].replace('USDT', '')} on SPOT"
            else:
                action = f"Need to SELL {abs(row['Futures Position'])} {row['Symbol'].replace('USDT', '')} on SPOT or close futures"
            
            print(f"  {row['Symbol']}: {action}")
            print(f"    Current: {row['Futures Direction']} {abs(row['Futures Position'])} futures, {row['Spot Quantity']} spot")
    
    return df

def quick_position_check():
    print("🔍 QUICK POSITION CHECK")
    print("-" * 40)
    
    try:
        # Get futures positions
        futures_pnls = futures_trader.get_position_pnls()
        
        if not futures_pnls:
            print("✅ No active futures positions")
            return
        
        print(f"📊 Active Futures Positions: {len(futures_pnls)}")
        
        for pos in futures_pnls:
            symbol = pos['symbol']
            position = float(pos['position'])
            pnl = float(pos['unrealized_pnl'])
            direction = pos['direction']
            
            print(f"  {symbol}: {direction} {abs(position):,.0f} | PnL: ${pnl:+.2f}")
        
        total_pnl = sum(float(pos['unrealized_pnl']) for pos in futures_pnls)
        print(f"\n💰 Total PnL: ${total_pnl:+.2f}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Main function to run
def show_position_matching():
    return match_futures_with_spot(futures_trader, spot_trader)

# Quick usage functions
def show_my_positions():
    df = show_position_matching()
    return df

# Even simpler version - just the essentials
def show_hedge_status():
    try:
        futures_pnls = futures_trader.get_position_pnls()
        account = spot_trader.client.get_account()
        
        print("🔗 HEDGE STATUS CHECK")
        print("-" * 50)
        
        for pos in futures_pnls:
            symbol = pos['symbol']
            futures_amt = float(pos['position'])
            direction = pos['direction']
            
            # Check spot
            asset = symbol.replace('USDT', '')
            spot_balance = next((b for b in account['balances'] if b['asset'] == asset), None)
            spot_qty = float(spot_balance['free']) + float(spot_balance['locked']) if spot_balance else 0
            
            # Status
            if direction == 'SHORT' and spot_qty > 0:
                status = "✅ HEDGED"
            elif direction == 'LONG' and spot_qty > 0:
                status = "⚠️ BOTH LONG"
            else:
                status = "❌ ORPHANED"
            
            print(f"{symbol}: {status}")
            print(f"  Futures: {direction} {abs(futures_amt):,.0f}")
            print(f"  Spot: {spot_qty:,.0f} {asset}")
            print()
            
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
# df = show_my_positions()
# quick_position_check()
# show_hedge_status()

In [234]:
# Show complete futures-to-spot matching
df = show_my_positions()

# Or just a quick status check
quick_position_check()

# Or even simpler - just hedge status
show_hedge_status()

FUTURES POSITIONS vs SPOT HOLDINGS
  Symbol  Futures Position Futures Direction  Spot Quantity                      Hedge Status Hedge Ratio  Futures PnL
LOKAUSDT            -68000             SHORT         231769 ✅ HEDGED (Short Perp + Long Spot)       29.3%       263.57
  OPUSDT             -1988             SHORT           5660 ✅ HEDGED (Short Perp + Long Spot)       35.1%        47.31

SUMMARY
Total Futures Positions: 2
Properly Hedged: 2
Orphaned (Need Spot): 0
Total Unrealized PnL: $310.88
🔍 QUICK POSITION CHECK
----------------------------------------
📊 Active Futures Positions: 2
  LOKAUSDT: SHORT 68,000 | PnL: $+263.57
  OPUSDT: SHORT 1,988 | PnL: $+47.31

💰 Total PnL: $+310.88
🔗 HEDGE STATUS CHECK
--------------------------------------------------
LOKAUSDT: ✅ HEDGED
  Futures: SHORT 68,000
  Spot: 231,769 LOKA

OPUSDT: ✅ HEDGED
  Futures: SHORT 1,988
  Spot: 5,660 OP



### ARCHIVE

In [ ]:
# # 1. First, let's check what spot positions you have
# def check_spot_holdings():
#     """Check what you actually bought on spot"""
#     client = Client(api_key_spot, api_secret_spot)
#     client.API_URL = 'https://testnet.binance.vision/api'
    
#     account_info = client.get_account()
#     balances = account_info['balances']
    
#     print("🔍 CURRENT SPOT HOLDINGS:")
#     print("=" * 50)
    
#     for asset in balances:
#         free_bal = float(asset['free'])
#         locked_bal = float(asset['locked'])
#         total_bal = free_bal + locked_bal
        
#         if total_bal > 0:
#             print(f"{asset['asset']}: Free={free_bal}, Locked={locked_bal}, Total={total_bal}")
    
#     return balances

# # 2. Get futures symbol info for precision
# def get_futures_symbol_info(symbol):
#     """Get futures symbol filters"""
#     try:
#         client = Client(api_key, api_secret)
#         client.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'
        
#         exchange_info = client.futures_exchange_info()
#         symbol_info = next((s for s in exchange_info['symbols'] if s['symbol'] == symbol), None)
        
#         if symbol_info:
#             filters = {}
#             for filter_info in symbol_info['filters']:
#                 if filter_info['filterType'] == 'PRICE_FILTER':
#                     filters['tickSize'] = float(filter_info['tickSize'])
#                     filters['minPrice'] = float(filter_info['minPrice'])
#                     filters['maxPrice'] = float(filter_info['maxPrice'])
#                 elif filter_info['filterType'] == 'LOT_SIZE':
#                     filters['stepSize'] = float(filter_info['stepSize'])
#                     filters['minQty'] = float(filter_info['minQty'])
#                     filters['maxQty'] = float(filter_info['maxQty'])
#                 elif filter_info['filterType'] == 'MIN_NOTIONAL':
#                     filters['minNotional'] = float(filter_info['notional'])
            
#             print(f"📊 {symbol} Futures Filters:")
#             for key, value in filters.items():
#                 print(f"  {key}: {value}")
            
#             return filters
#         else:
#             print(f"❌ Symbol {symbol} not found")
#             return None
            
#     except Exception as e:
#         print(f"❌ Error getting symbol info: {e}")
#         return None

# # 3. Fix quantity precision function
# import math
# from decimal import Decimal, ROUND_DOWN

# def fix_futures_precision(price, quantity, symbol):
#     """Fix price and quantity to futures exchange precision"""
    
#     filters = get_futures_symbol_info(symbol)
#     if not filters:
#         return price, quantity
    
#     # Fix price precision
#     tick_size = filters['tickSize']
#     price_precision = int(round(-math.log10(tick_size)))
#     fixed_price = round(price / tick_size) * tick_size
#     fixed_price = round(fixed_price, price_precision)
    
#     # Fix quantity precision  
#     step_size = filters['stepSize']
#     qty_precision = int(round(-math.log10(step_size)))
#     fixed_quantity = math.floor(quantity / step_size) * step_size
#     fixed_quantity = round(fixed_quantity, qty_precision)
    
#     # Check minimum quantity
#     min_qty = filters['minQty']
#     if fixed_quantity < min_qty:
#         print(f"⚠️ Quantity {fixed_quantity} below minimum {min_qty}")
#         return None, None
    
#     # Check minimum notional
#     notional = fixed_price * fixed_quantity
#     min_notional = filters.get('minNotional', 5)
#     if notional < min_notional:
#         print(f"⚠️ Notional ${notional} below minimum ${min_notional}")
#         return None, None
    
#     print(f"✅ Fixed precision for {symbol}:")
#     print(f"  Price: {price} → {fixed_price}")
#     print(f"  Quantity: {quantity} → {fixed_quantity}")
#     print(f"  Notional: ${notional}")
    
#     return fixed_price, fixed_quantity

# # 4. Test a single futures order with proper precision
# def test_futures_order_with_precision():
#     """Test futures order with proper precision"""
    
#     symbol = 'LOKAUSDT'  # From your logs
#     side = 'SELL'
#     test_price = 0.0567  # Recent price from your logs
#     test_quantity = 4735.0  # From your position
    
#     print(f"🧪 Testing {symbol} futures order...")
    
#     # Fix precision
#     fixed_price, fixed_quantity = fix_futures_precision(test_price, test_quantity, symbol)
    
#     if fixed_price is None:
#         print("❌ Cannot fix precision")
#         return
    
#     try:
#         # Test the order
#         futures_trader = FuturesTrader(api_key, api_secret)
        
#         order = futures_trader.client.futures_create_order(
#             symbol=symbol,
#             side=side,
#             type='LIMIT',
#             timeInForce='GTC',  # Changed to GTC
#             quantity=fixed_quantity,
#             price=fixed_price
#         )
        
#         print(f"✅ Test order successful!")
#         print(f"Order ID: {order.get('orderId')}")
#         print(f"Status: {order.get('status')}")
        
#         return order
        
#     except Exception as e:
#         print(f"❌ Test order failed: {e}")
#         return None

# # 5. Close your orphaned spot positions
# def close_orphaned_spot_positions():
#     """Close spot positions that don't have corresponding futures positions"""
    
#     # Get current spot holdings
#     balances = check_spot_holdings()
    
#     # Get current futures positions
#     futures_trader = FuturesTrader(api_key, api_secret)
#     futures_positions = futures_trader.get_position_pnls()
#     futures_symbols = {p['symbol'] for p in futures_positions}
    
#     print(f"\n🔄 Futures positions: {futures_symbols}")
    
#     # Find orphaned spot positions
#     orphaned = []
#     for asset in balances:
#         if asset['asset'] == 'USDT':
#             continue
            
#         total = float(asset['free']) + float(asset['locked'])
#         if total > 0:
#             symbol = f"{asset['asset']}USDT"
#             if symbol not in futures_symbols:
#                 orphaned.append({
#                     'asset': asset['asset'],
#                     'symbol': symbol,
#                     'quantity': total
#                 })
    
#     print(f"\n⚠️ Orphaned spot positions: {len(orphaned)}")
#     for pos in orphaned:
#         print(f"  {pos['asset']}: {pos['quantity']}")
    
#     # Ask if you want to close them
#     if orphaned:
#         print(f"\n💡 You should close these orphaned positions to recover your USDT")
#         print(f"Run: close_spot_position('SYMBOL', quantity) for each")
    
#     return orphaned

# def close_spot_position(symbol, quantity):
#     """Close a specific spot position"""
#     try:
#         spot_trader = SpotTrader(api_key_spot, api_secret_spot)
        
#         order = spot_trader.place_market_order(symbol, 'SELL', quantity)
#         print(f"✅ Closed {quantity} {symbol}: {order}")
        
#         return order
#     except Exception as e:
#         print(f"❌ Failed to close {symbol}: {e}")
#         return None

# # Run these checks
# print("🔍 DIAGNOSING YOUR TRADING ISSUES...")
# print("=" * 60)

# # Check what you actually have
# balances = check_spot_holdings()

# # Check futures positions
# print(f"\n🔍 CURRENT FUTURES POSITIONS:")
# futures_trader = FuturesTrader(api_key, api_secret)
# pnls = futures_trader.get_position_pnls()
# for p in pnls:
#     print(f"  {p['symbol']}: {p['position']} (PnL: ${p['unrealized_pnl']})")

# # Find orphaned positions
# orphaned = close_orphaned_spot_positions()

# # Test futures precision
# print(f"\n🧪 TESTING FUTURES PRECISION...")
# test_futures_order_with_precision()

# print(f"\n💡 NEXT STEPS:")
# print(f"1. Close orphaned spot positions to recover USDT")
# print(f"2. Update your trade function with proper precision")
# print(f"3. Test with small amounts first")

In [ ]:
# # IMMEDIATE RECOVERY PLAN

# def emergency_recovery():
#     """Emergency recovery of your positions"""
#     print("🚨 EMERGENCY RECOVERY MODE")
#     print("=" * 50)
    
#     # 1. Get current state
#     print("1️⃣ Checking current positions...")
#     balances = check_spot_holdings()
    
#     futures_trader = FuturesTrader(api_key, api_secret)
#     futures_positions = futures_trader.get_position_pnls()
    
#     print(f"\nFutures positions:")
#     for pos in futures_positions:
#         print(f"  {pos['symbol']}: {pos['position']} (PnL: ${pos['unrealized_pnl']:.2f})")
    
#     # 2. Calculate what you should have based on trade log
#     print(f"\n2️⃣ Analyzing trade log...")
#     trade_df = pd.DataFrame(trade_log)
#     if not trade_df.empty:
#         print(f"Trade log shows {len(trade_df)} trades:")
#         for _, trade in trade_df.iterrows():
#             print(f"  {trade['symbol']}: {trade['side']}")
            
#             # Check if this trade is properly hedged
#             symbol = trade['symbol']
#             spot_side = trade['spot_leg']['side']
#             futures_side = trade['futures_leg']['side']
#             quantity = trade['spot_leg']['quantity']
            
#             print(f"    Should have: {spot_side} {quantity} spot, {futures_side} {quantity} futures")
    
#     # 3. Find mismatches and orphaned positions
#     print(f"\n3️⃣ Finding orphaned positions...")
    
#     # Get futures symbols
#     futures_symbols = {pos['symbol'] for pos in futures_positions}
    
#     # Check spot holdings
#     orphaned_spot = []
#     for asset in balances:
#         if asset['asset'] == 'USDT':
#             continue
            
#         total = float(asset['free']) + float(asset['locked'])
#         if total > 0:
#             symbol = f"{asset['asset']}USDT"
#             if symbol not in futures_symbols:
#                 orphaned_spot.append({
#                     'asset': asset['asset'],
#                     'symbol': symbol,
#                     'quantity': total,
#                     'estimated_value': total * 0.02  # Rough estimate
#                 })
    
#     # Check futures positions without spot hedge
#     orphaned_futures = []
#     spot_assets = {asset['asset'] for asset in balances if float(asset['free']) + float(asset['locked']) > 0}
    
#     for pos in futures_positions:
#         base_asset = pos['symbol'].replace('USDT', '')
#         if base_asset not in spot_assets:
#             orphaned_futures.append(pos)
    
#     print(f"\n📊 ORPHANED POSITIONS SUMMARY:")
#     print(f"Orphaned spot positions: {len(orphaned_spot)}")
#     total_spot_value = sum(pos['estimated_value'] for pos in orphaned_spot)
#     print(f"Estimated value locked in spot: ~${total_spot_value:.2f}")
    
#     for pos in orphaned_spot:
#         print(f"  {pos['symbol']}: {pos['quantity']}")
    
#     print(f"\nOrphaned futures positions: {len(orphaned_futures)}")
#     for pos in orphaned_futures:
#         print(f"  {pos['symbol']}: {pos['position']} (PnL: ${pos['unrealized_pnl']:.2f})")
    
#     return orphaned_spot, orphaned_futures

# def close_all_orphaned_positions():
#     """Close all orphaned positions to recover capital"""
    
#     orphaned_spot, orphaned_futures = emergency_recovery()
    
#     if not orphaned_spot and not orphaned_futures:
#         print("✅ No orphaned positions found!")
#         return
    
#     print(f"\n🔄 CLOSING ORPHANED POSITIONS...")
    
#     # Close orphaned spot positions
#     spot_trader = SpotTrader(api_key_spot, api_secret_spot)
#     futures_trader = FuturesTrader(api_key, api_secret)
    
#     recovered_usdt = 0
    
#     for pos in orphaned_spot:
#         try:
#             print(f"\n🔄 Closing spot position: {pos['symbol']} ({pos['quantity']})")
            
#             order = spot_trader.place_market_order(pos['symbol'], 'SELL', pos['quantity'])
            
#             if order and order.get('status') == 'FILLED':
#                 executed_qty = float(order.get('executedQty', 0))
#                 avg_price = float(order.get('cummulativeQuoteOrderQty', 0)) / executed_qty if executed_qty > 0 else 0
#                 recovered = executed_qty * avg_price
#                 recovered_usdt += recovered
                
#                 print(f"✅ Sold {executed_qty} {pos['asset']} for ${recovered:.2f}")
#             else:
#                 print(f"❌ Failed to sell {pos['symbol']}")
                
#         except Exception as e:
#             print(f"❌ Error closing {pos['symbol']}: {e}")
    
#     # Close orphaned futures positions
#     for pos in orphaned_futures:
#         try:
#             symbol = pos['symbol']
#             position_amt = float(pos['position'])
            
#             if position_amt == 0:
#                 continue
                
#             # Determine closing side
#             close_side = 'BUY' if position_amt < 0 else 'SELL'
#             close_quantity = abs(position_amt)
            
#             print(f"\n🔄 Closing futures position: {symbol} ({position_amt})")
            
#             order = futures_trader.place_market_order(symbol, close_side, close_quantity)
            
#             if order and order.get('status') == 'FILLED':
#                 print(f"✅ Closed {symbol} futures position")
#                 print(f"   Realized PnL: ${pos['unrealized_pnl']:.2f}")
#             else:
#                 print(f"❌ Failed to close {symbol} futures position")
                
#         except Exception as e:
#             print(f"❌ Error closing futures {pos['symbol']}: {e}")
    
#     print(f"\n💰 RECOVERY SUMMARY:")
#     print(f"Estimated USDT recovered from spot: ${recovered_usdt:.2f}")
#     print(f"Check your balances now!")

# def fix_trade_function_precision():
#     """Show how to fix the precision in your trade function"""
    
#     print(f"\n🔧 FIXING YOUR TRADE FUNCTION:")
#     print("=" * 50)
    
#     print("""
# Add this to your trade_top_opportunity function:

# # BEFORE placing futures orders, add this:
# def place_futures_order_with_precision(symbol, side, quantity, price):
#     \"\"\"Place futures order with proper precision\"\"\"
    
#     # Get futures filters
#     filters = get_futures_symbol_info(symbol)
#     if not filters:
#         logging.error(f"Cannot get filters for {symbol}")
#         return None
    
#     # Fix precision
#     tick_size = filters['tickSize']
#     step_size = filters['stepSize']
    
#     # Fix price
#     fixed_price = round(price / tick_size) * tick_size
#     price_precision = int(round(-math.log10(tick_size)))
#     fixed_price = round(fixed_price, price_precision)
    
#     # Fix quantity
#     fixed_quantity = math.floor(quantity / step_size) * step_size
#     qty_precision = int(round(-math.log10(step_size)))
#     fixed_quantity = round(fixed_quantity, qty_precision)
    
#     # Check minimums
#     if fixed_quantity < filters['minQty']:
#         logging.error(f"Quantity {fixed_quantity} below minimum {filters['minQty']}")
#         return None
    
#     notional = fixed_price * fixed_quantity
#     min_notional = filters.get('minNotional', 5)
#     if notional < min_notional:
#         logging.error(f"Notional ${notional} below minimum ${min_notional}")
#         return None
    
#     logging.info(f"Fixed precision: {quantity}→{fixed_quantity}, {price}→{fixed_price}")
    
#     # Place order with fixed values
#     return futures_trader.place_limit_order(symbol, side, fixed_quantity, fixed_price)

# # REPLACE your futures order calls with:
# # futures_order = place_futures_order_with_precision(futures_symbol, SIDE_SELL, quantity, entry_price)
# """)

# # Run the emergency recovery
# print("🚨 STARTING EMERGENCY DIAGNOSIS AND RECOVERY...")
# print("This will show you exactly what positions you have and help recover your USDT")
# print("=" * 80)

# # Run diagnosis first
# orphaned_spot, orphaned_futures = emergency_recovery()

# # Show next steps
# print(f"\n🎯 RECOMMENDED ACTIONS:")
# print(f"1. Run: close_all_orphaned_positions() to recover your USDT")
# print(f"2. Fix your trade function precision (code shown above)")
# print(f"3. Test with small amounts before resuming trading")

# print(f"\n⚠️ CURRENT USDT BALANCE WARNING:")
# print(f"You started with ~$10,000 and now have only ~$772 in spot USDT")
# print(f"Most of your capital is locked in orphaned spot positions")
# print(f"Recovery should get you back to ~$8,000-9,000 depending on price movements")
# # Run this to see exactly what you have
# emergency_recovery()
# # This will close all orphaned spot positions and recover your USDT
# close_all_orphaned_positions()

In [ ]:
# # Run this to see exactly what you have
# emergency_recovery()

# # This will close all orphaned spot positions and recover your USDT
# close_all_orphaned_positions()